##  Route-Level Aggregation & KPI Calculation

This notebook aggregates cleaned data from flights, tickets, and airport codes into a unified route-level dataset. It then computes key profitability metrics for each unique airline route.

---

###  Data Merging Strategy

 load the cleaned datasets from `/data/cleaned/`:
- `flights_cleaned.csv`
- `tickets_cleaned.csv`
- `airports_cleaned.csv`

To ensure flexibility and scalability, we used a modular merge utility (`merge_utils.py`) that supports:
-  `Pandas` (default, used here)
-  `Dask` (for out-of-core processing)
-  `Polars` (for high-performance columnar processing)

I used the function:

merged = merge_flights_tickets_airports_pandas(flights_grouped, tickets_grouped, a)

This performs the following:

Merges flights_grouped and tickets_grouped on ROUTE_KEY + CARRIER.

Extracts AIRPORT_A and AIRPORT_B from ROUTE_KEY.

Joins metadata from both airports (like size, elevation, and coordinates).

### Flights Aggregation
group the cleaned flights data by:

Carrier
Route (ORIGIN-DESTINATION, symmetric handling)
City names

Then compute:
AVG_DEP_DELAY, AVG_ARR_DELAY
AVG_OCCUPANCY_RATE
MEAN_DISTANCE
TOTAL_FLIGHTS across both directions (e.g., ATL → JFK and JFK → ATL)


### Tickets Aggregation
group the cleaned ticket data by:

ROUTE_KEY (symmetric)
REPORTING_CARRIER

Then compute:
TOTAL_PASSENGERS per route
AVG_ROUNDTRIP_FARE_PER_PASSENGER


### KPI Calculations
After merging, the enriched route-level dataset (final_df) is used to compute financial KPIs using a business rules-based model.

#### Revenue Estimation
ESTIMATED_PASSENGERS = Occupancy × Plane Capacity × Total Flights

TICKET_REVENUE = ESTIMATED_PASSENGERS × Avg Fare

BAGGAGE_REVENUE = 50% × ESTIMATED_PASSENGERS × $70

TOTAL_REVENUE = TICKET_REVENUE + BAGGAGE_REVENUE

#### Cost Estimation
ROUND_TRIP_DISTANCE = ONE_WAY_DISTANCE × 2

MILEAGE_COST = Distance × Flights × ($8 + $1.18)

AIRPORT_FEES = (Medium or Large) × 2 × Flights

DELAY_COST = Delay beyond 15 min × $75/min × Flights

TOTAL_COST = Mileage + Airport + Delay

#### Profitability
PROFIT = TOTAL_REVENUE - TOTAL_COST

All final values are rounded to 2 decimals for readability.

#### Output
The final aggregated dataframe (final_df) contains all columns necessary for:
Ranking profitable routes
Filtering by airport size
Plotting costs vs. revenue

In [5]:
# aggregate_and_kpis.py
import pandas as pd
import numpy as np
import os 
import sys 
import warnings
warnings.filterwarnings("ignore")
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(project_root, 'src')

if src_path not in sys.path:
    sys.path.insert(0, src_path)

print("src/ path added:", src_path)

from merge_utils import ( merge_and_add_airport_details,merge_flights_tickets_airports,
                         merge_flights_tickets_airports_dask,merge_flights_tickets_airports_pandas,
                         merge_flights_tickets_airports_polars
)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_dir = os.path.join(project_root, 'data', 'cleaned')
airports_file_path = os.path.join(data_dir, 'airports_cleaned.csv')
tickets_file_path = os.path.join(data_dir, 'tickets_cleaned.csv')
flights_file_path = os.path.join(data_dir, 'flights_cleaned.csv')

airports_cols = [ 'TYPE', 'NAME', 'ELEVATION_FT',
       'MUNICIPALITY', 'IATA_CODE', 'COORDINATES']

flights_cols = [ 'FL_DATE', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN', 'ORIGIN_CITY_NAME', 'DEST_AIRPORT_ID',
       'DESTINATION', 'DEST_CITY_NAME', 'DEP_DELAY', 'ARR_DELAY', 'CANCELLED',
       'AIR_TIME', 'DISTANCE', 'OCCUPANCY_RATE']

tickets_cols = [ 'ITIN_ID', 'YEAR', 'QUARTER', 'ORIGIN', 'ORIGIN_COUNTRY',
       'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM', 'ROUNDTRIP', 'REPORTING_CARRIER',
       'PASSENGERS', 'ITIN_FARE', 'DESTINATION']

# === Load Cleaned Datasets ===
a = pd.read_csv(airports_file_path)
a=a[airports_cols]
t = pd.read_csv(tickets_file_path)
t = t[tickets_cols]
f = pd.read_csv(flights_file_path, low_memory=False)
f = f[flights_cols]


# === Preprocessing ===
f['FL_DATE'] = pd.to_datetime(f['FL_DATE'], errors='coerce')
f['YEAR'] = f['FL_DATE'].dt.year
f['MONTH'] = f['FL_DATE'].dt.month

# === Flights Aggregation ===
flights_grouped = f.groupby(['OP_CARRIER','ORIGIN', 'DESTINATION', 'ORIGIN_CITY_NAME', 'DEST_CITY_NAME']).agg({
    'DISTANCE': 'mean',
    'OCCUPANCY_RATE': 'mean',
    'DEP_DELAY': 'mean',
    'ARR_DELAY': 'mean',
    'TAIL_NUM': 'count'
}).reset_index().rename(columns={'TAIL_NUM': 'TOTAL_FLIGHTS'})

flights_grouped['ROUTE_KEY'] = flights_grouped.apply(
    lambda row: '-'.join(sorted([row['ORIGIN'], row['DESTINATION']])), axis=1
)

flights_grouped['ROUTE_NAME_CITY'] = flights_grouped.apply(
    lambda row: ' -> '.join(sorted([row['ORIGIN_CITY_NAME'], row['DEST_CITY_NAME']])), axis=1
)

flights_grouped = flights_grouped.groupby(['ROUTE_KEY','OP_CARRIER', 'ROUTE_NAME_CITY']).agg({
    'DISTANCE': 'mean',
    'OCCUPANCY_RATE': 'mean',
    'DEP_DELAY': 'mean',
    'ARR_DELAY': 'mean',
    'TOTAL_FLIGHTS': 'sum'
}).reset_index()

# === Tickets Aggregation ===
t['ROUTE_KEY'] = t.apply(lambda row: '-'.join(sorted([row['ORIGIN'], row['DESTINATION']])), axis=1)
tickets_grouped = t.groupby(['ROUTE_KEY', 'REPORTING_CARRIER']).agg({
    'PASSENGERS': 'sum',
    'ITIN_FARE': 'mean'
}).reset_index()


#pandas version
merged = merge_flights_tickets_airports_pandas(flights_grouped, tickets_grouped, a)
# polars version
#merged = merge_flights_tickets_airports_polars(flights_grouped, tickets_grouped, a)

pd.set_option('display.max_columns', None)
merged = merged.rename(columns={
    'ROUTE_KEY': 'ROUTE',
    'OP_CARRIER': 'CARRIER',
    'DISTANCE': 'ONE_WAY_DISTANCE_MILES',
    'OCCUPANCY_RATE': 'AVG_OCCUPANCY_RATE',
    'DEP_DELAY': 'AVG_DEP_DELAY_MINUTES',
    'ARR_DELAY': 'AVG_ARR_DELAY_MINUTES',
    'TOTAL_FLIGHTS': 'TOTAL_FLIGHTS',
    'REPORTING_CARRIER': 'CARRIER_FROM_TICKETS',
    'PASSENGERS': 'TOTAL_PASSENGERS',
    'ITIN_FARE': 'AVG_ROUNDTRIP_FARE_PER_PASSENGER',
    'AIRPORT_A': 'AIRPORT_CODE_A',
    'AIRPORT_B': 'AIRPORT_CODE_B',
    'AIRPORT_A_TYPE': 'AIRPORT_A_SIZE',
    'AIRPORT_B_TYPE': 'AIRPORT_B_SIZE',
    'NAME_x': 'AIRPORT_A_NAME',
    'NAME_y': 'AIRPORT_B_NAME',
    'ELEVATION_FT_x': 'AIRPORT_A_ELEVATION_FT',
    'ELEVATION_FT_y': 'AIRPORT_B_ELEVATION_FT',
    'MUNICIPALITY_x': 'AIRPORT_A_CITY',
    'MUNICIPALITY_y': 'AIRPORT_B_CITY',
    'COORDINATES_x': 'AIRPORT_A_COORDINATES',
    'COORDINATES_y': 'AIRPORT_B_COORDINATES'
})


# Airplane purchase cost (per airplane)
airplane_cost = 90_000_000  # $90 million per airplane

# Cost assumptions per mile
fuel_oil_maintenance_crew_per_mile = 8.00  # in USD
depreciation_insurance_other_per_mile = 1.18  # in USD

# Airport fees
medium_airport_fee = 5_000  # USD per landing
large_airport_fee = 10_000  # USD per landing

# Delay cost
delay_cost_per_minute = 75  # USD per minute
delay_free_minutes = 15  # First 15 minutes free for each departure/arrival

# Revenue assumptions
plane_capacity = 200  # passengers per flight
baggage_fee_per_bag_one_way = 35  # USD
round_trip_baggage_fee_per_bag = baggage_fee_per_bag_one_way * 2  # USD for round trip

# Baggage check percentage
baggage_check_percentage = 0.50  # 50% of passengers

# Total operational cost per mile (for later ease of use)
total_operational_cost_per_mile = fuel_oil_maintenance_crew_per_mile + depreciation_insurance_other_per_mile

# Display variables to verify
print(f"Airplane Cost: ${airplane_cost:,}")
print(f"Total operational cost per mile: ${total_operational_cost_per_mile:.2f}")
print(f"Medium Airport Fee: ${medium_airport_fee}")
print(f"Large Airport Fee: ${large_airport_fee}")
print(f"Delay cost per minute (after {delay_free_minutes} min): ${delay_cost_per_minute}")
print(f"Plane capacity: {plane_capacity} passengers")
print(f"Baggage fee per round trip bag: ${round_trip_baggage_fee_per_bag}")
print(f"Baggage check percentage: {baggage_check_percentage*100:.0f}%")

final_df = merged.copy()

# Step 1: Estimate passengers based on occupancy rate and total flights
final_df['ESTIMATED_PASSENGERS'] = final_df['AVG_OCCUPANCY_RATE'] * plane_capacity * final_df['TOTAL_FLIGHTS']

# Step 2: Revenue formulas (based on estimated passengers)

# Ticket revenue (estimated passengers × average round trip fare)
final_df['TICKET_REVENUE'] = final_df['ESTIMATED_PASSENGERS'] * final_df['AVG_ROUNDTRIP_FARE_PER_PASSENGER']

# Baggage revenue (50% of estimated passengers check 1 bag per round trip)
final_df['BAGGAGE_REVENUE'] = final_df['ESTIMATED_PASSENGERS'] * baggage_check_percentage * round_trip_baggage_fee_per_bag

# Total Revenue
final_df['TOTAL_REVENUE'] = final_df['TICKET_REVENUE'] + final_df['BAGGAGE_REVENUE']

# Step 3: Operational costs

# Round trip distance = outbound + return
final_df['ROUND_TRIP_DISTANCE_MILES'] = final_df['ONE_WAY_DISTANCE_MILES'] * 2

# Mileage cost = distance × flights × per mile costs
final_df['MILEAGE_COST'] = final_df['ROUND_TRIP_DISTANCE_MILES'] * total_operational_cost_per_mile * final_df['TOTAL_FLIGHTS']

# Airport fees per round trip flight
def airport_fee(row):
    fee_a = medium_airport_fee if row['AIRPORT_A_SIZE'] == 'medium_airport' else large_airport_fee
    fee_b = medium_airport_fee if row['AIRPORT_B_SIZE'] == 'medium_airport' else large_airport_fee
    return (fee_a + fee_b) * row['TOTAL_FLIGHTS']

final_df['AIRPORT_FEES'] = final_df.apply(airport_fee, axis=1)

# Delay cost (departure and arrival combined)
def delay_cost(delay):
    excess = max(delay - delay_free_minutes, 0)
    return excess * delay_cost_per_minute

final_df['DELAY_COST'] = final_df.apply(
    lambda row: (delay_cost(row['AVG_DEP_DELAY_MINUTES']) + delay_cost(row['AVG_ARR_DELAY_MINUTES'])) * row['TOTAL_FLIGHTS'],
    axis=1
)

# Step 4: Total cost and profit
final_df['TOTAL_COST'] = final_df['MILEAGE_COST'] + final_df['AIRPORT_FEES'] + final_df['DELAY_COST']
final_df['PROFIT'] = final_df['TOTAL_REVENUE'] - final_df['TOTAL_COST']

cols_to_round = ['TICKET_REVENUE', 'BAGGAGE_REVENUE', 'TOTAL_REVENUE', 
                 'MILEAGE_COST', 'AIRPORT_FEES', 'DELAY_COST', 'TOTAL_COST', 'PROFIT']
final_df[cols_to_round] = final_df[cols_to_round].round(2)

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_dir = os.path.join(project_root, 'data', 'cleaned')
final_df_file_path = os.path.join(data_dir, 'final_df.csv')
final_df.to_csv(final_df_file_path, index=None)

print("Completed")


✅ src/ path added: /Users/nyzy/nitzmali/capital_one_data_challenge/capital_one_data_challenge/src
Airplane Cost: $90,000,000
Total operational cost per mile: $9.18
Medium Airport Fee: $5000
Large Airport Fee: $10000
Delay cost per minute (after 15 min): $75
Plane capacity: 200 passengers
Baggage fee per round trip bag: $70
Baggage check percentage: 50%
